In [5]:
# Import numpy library for use in ...
import numpy as np

# Function info here ...
def import_fa(file_name, return_labels=False):
    print("Importing "+file_name)
    # List storage for file contents (sequences) and sequene labels
    sequences = []
    labels = []
    # Try to open the file as read only
    try:
        file = open(file_name, "r")
    # If the file would not open, print error message and exit
    except:
        print("\nError: Could not open file")
        print("File name given: "+file_name+"\n")
        raise SystemExit
    # Iterate over all the lines in the file
    for line in file.readlines():
        # If this character is at the start of the line, it is a sequence label
        if line[0] == ">":
            # If the argument 'return_labels' is True, append the label to a list
            if return_labels:
                labels.append(str(line)[1:-1])
            else:
                continue
        # Append the sequence to the list of sequences
        sequences.append(str(line)[:-1])
    print("    "+str(len(sequences))+" sequences")
    # If the argument 'return_labels' is True, return arrays of the sequences and labels
    if return_labels:
        return np.array(sequences), np.array(labels)
    # If False, only return an array of the sequences
    else:
        return np.array(sequences)

# Function info here ...
def raw_to_features(data,balance=False,resample=1.0):
    print("\nConverting to features:")
    n = len(data)
    # Find basic data stats for function use:
    #   the max length sequence in the data
    #   the number of classes
    #   count of each class
    max_seq_len = 0
    class_counts = [0]
    num_classes = -1
    # Iterate over all sequences
    for line in data:
        # Look out for a new class
        if "class" in line:
            class_counts.append(1)
            num_classes += 1
            continue
        # Update max sequence length
        if len(line) > max_seq_len:
            max_seq_len = len(line)
        # Update class counts
        class_counts[num_classes+1] += 1
    
    # Since this was -1 based, increment to convert to 0 based count
    num_classes += 1
    
    # Print stats found above
    print("  Num Classes:", num_classes)
    print("  Class Counts:")
    frmt = "{:>7}"*num_classes
    print(" "+frmt.format(*[i for i in range(num_classes)]))
    print("   ",end="")
    frmt = "{:>7}"*num_classes
    print(" "+frmt.format(*[class_counts[i] for i in range(1,num_classes+1)]))

    # Split data by class for resampling
    data_by_class = []
    for i in range(num_classes):
        data_by_class.append( data[class_counts[i]:class_counts[i+1]] )
    
    # Instantiate storage of features and labels
    X = []
    y = []
    
    # Init variables for resampling
    total_per_class = 0
    num_samples = n
    # If resampling:
    if resample != 1.0:
        # Calculate the size of the resampled dataset
        num_samples = int(n*resample)
        # If balanced classs coutns are required:
        if balance:
            # Calculate a balanced total number of instances per class
            total_per_class = int(num_samples/num_classes)
            # Pull instances
            for i in range(num_classes):
                for j in range(1,total_per_class+1): 
                    X.append( data_by_class[i][j] )
            # Pull labels
            for i in range(num_classes):
                for j in range(total_per_class):
                    y.append(i)
        # If imbalanced class counts are acceptable
        else:
            # Calculate the total number of instances per class in the same ratio as class counts
            total_per_class = [int(num_samples*class_counts[i+1]/n) for i in range(num_classes)]
            # Pull instances
            for i in range(num_classes):
                for j in range(total_per_class[i]+1,total_per_class[i+1]+1):
                    X.append( data_by_class[i][j] )
            # Pull labels
            for i in range(num_classes):
                for j in range(total_per_class):
                    y.append(i)
        print("  Resampling:")
        print("    Total samples:",num_samples)
        print("    Total/class  :",total_per_class)
        print()
    # If not resampling
    # TODO:  Complete - could the above be used instead of conditional?
    else:
        total_per_class = int(n/num_classes)
    
    # Convert from strings to arrays and pad the sequences with zeros to 
    # the length of the maximum length sequence; this is the number of features
    for i in range(len(X)):
        # Instantiate new array of zeros
        new_instance = ['0' for i in range(max_seq_len)]
        # Fill in sequence
        for j in range(len(X[i])):
            new_instance[j] = X[i][j]
        # Store new sequence in features store X
        X[i] = new_instance

    # Return features and labels
    return X,y

In [6]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU

In [7]:
data = import_fa("data/pos_neg-unip.fa")
X, y = raw_to_features(data, balance=True, resample=0.01)

y_one_hot = to_categorical(y)

from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y_one_hot, test_size=0.2, random_state=0)

# Number of data instances
n = len(train_X)
# Number of features
m = len(train_X[0])


Importing data/pos_neg-unip.fa
    30050 sequences

Converting to features:
  Num Classes: 2
  Class Counts:
       0      1
       3259  26791
  Resampling:
    Total samples: 300
    Total/class  : 150



In [8]:
# Convert Numpy array to Tensor
train_X = tf.convert_to_tensor(train_X)
train_X = tf.reshape( train_X, (len(train_X), len(train_X[0]), 1) )
print(tf.shape(train_X))

tf.Tensor([ 240 5537    1], shape=(3,), dtype=int32)


2021-12-03 18:39:55.982833: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-03 18:39:55.983261: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-03 18:39:55.983340: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-03 18:39:55.983411: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-03 18:39:55.983481: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [9]:

# Convert Numpy array to Tensor
train_y = tf.convert_to_tensor(train_y)
train_y = tf.reshape(train_y, (len(train_y), len(train_y[0]),1))
print(tf.shape(train_y))

tf.Tensor([240   2   1], shape=(3,), dtype=int32)


In [10]:
batch_size = 64
epochs = 20
num_classes = 2

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(1,3),activation='linear',input_shape=(n,m,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=2,padding='same'))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    loss=keras.losses.categorical_crossentropy, 
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 240, 5537, 32)     128       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 240, 5537, 32)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 120, 2769, 32)    0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 10632960)          0         
                                                                 
 dense (Dense)               (None, 2)                 21265922  
                                                                 
Total params: 21,266,050
Trainable params: 21,266,050
Non-trainable params: 0
____________________________________________

In [11]:
print(np.shape(train_X))
model = model.fit(train_X, train_y, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(test_X, test_y))

(240, 5537, 1)
Epoch 1/20


ValueError: in user code:

    File "/home/chris/.local/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/chris/.local/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/chris/.local/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/chris/.local/lib/python3.8/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/home/chris/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/chris/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 263, in assert_input_compatibility
        raise ValueError(f'Inpu